## 번역가는 대화에도 능하다 [프로젝트]

- 챗봇 훈련데이터를 위한 전처리와 augmentation이 적절히 수행되어 3만개 가량의 훈련데이터셋이 구축되었다.
- 과적합을 피할 수 있는 하이퍼파라미터 셋이 적절히 제시되었다.
- 주어진 예문을 포함하여 챗봇에 던진 질문에 적절히 답하는 사례가 제출되었다.

Step 1. 데이터 다운로드
- https://github.com/songys/Chatbot_data
- 읽어 온 데이터의 질문과 답변을 각각 questions, answers 변수

In [1]:
import numpy as np
import tensorflow as tf
import nltk
import gensim


In [2]:
import os
import pandas as pd

path = os.getenv("HOME") + '/aiffel/Aiffel_online_Quest/GoingDeeper06'
df = pd.read_csv(path+'/ChatbotData.csv')
df.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11823 entries, 0 to 11822
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Q       11823 non-null  object
 1   A       11823 non-null  object
 2   label   11823 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 277.2+ KB


In [4]:
# 중복 제거
duplicated_q = list(df[df['Q'].duplicated()].index)
duplicated_q += list(df[df['A'].duplicated()].index)
df = df.drop(set(duplicated_q), axis=0)

In [217]:
question = df['Q'].values
answer = df['A'].values


print("question",question[0])
print("answer", answer[0])


question 12시 땡!
answer 하루가 또 가네요.


Step 2. 데이터 정제
-  소문자 변환
- 정규식을 활용해서 특수문자 제외
- 중복되는 문장은 데이터에서 제외 ( 위에서 이미 제외 )

In [218]:
# 중복제거 
# 데이터 전처리 함수 재정의
import re
def preprocess_sentence(sentence):
    
    # 소문자, 특수 문자 제외
    sentence = sentence.lower().strip()
    sentence = re.sub(r"[?.!,]+", "", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    #sentence = re.sub(r"[^a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣?.!,]+", " ", sentence)
    
    return sentence.strip()


In [256]:
question = [preprocess_sentence(q) for q in question]
answer = [preprocess_sentence(a) for a in answer]


print(answer[0])
print(question[0])

하루가 또 가네요
12시 땡


Step 3. 데이터 토큰화
- KoNLPy 사용
- build_corpus() 함수를 구현
    - 소스 문장 데이터와 타겟 문장 데이터를 입력
    - 데이터를 앞서 정의한 preprocess_sentence() 함수로 정제
    - 토큰화는 전달받은 토크나이즈 함수를 사용합니다. 이번엔 mecab.morphs
    - 토큰의 개수가 일정 길이 이상인 문장은 데이터에서 제외


In [257]:
from konlpy.tag import Mecab

def build_corpus(question, answer):
    # 데이터 정제
    ques = [preprocess_sentence(q) for q in question]
    ans = [preprocess_sentence(a) for a in answer]
    
    # 토크나이저로 토큰화
    mecab = Mecab()
    tokenized_question, tokenized_answer = [], []
    for q, a in zip(ques, ans):
        tokenized_question.append(mecab.morphs(q))
        tokenized_answer.append(mecab.morphs(a))

    # 길이 필터링
    filtered_question = [q for q in tokenized_question if len(q) < 50 and len(q)>2]
    filtered_answer = [a for a, q in zip(tokenized_answer, tokenized_question) if len(q) < 50 and len(q)>2]
    
    return filtered_question, filtered_answer
    
# 추상적인 사고 == 노이즈 == 창의력

In [258]:
q_data,a_data = build_corpus(question,answer)

In [259]:
print(len(q_data))
print(len(a_data))

7378
7378


In [260]:
print(q_data[0])
print(a_data[0])

['12', '시', '땡']
['하루', '가', '또', '가', '네요']


Step 4. Augmentation
-  Lexical Substitution을 실제로 적용
-  한국어로 사전 훈련된 Embedding 모델을 다운로드
- Korean (w) 가 Word2Vec으로 학습한 모델이며 용량도 적당하므로 사이트에서 Korean (w)를 찾아 다운로드하고, ko.bin 파일 , 다운로드한 모델을 활용해 데이터를 Augmentation 
- Augmentation된 que_corpus 와 원본 ans_corpus 가 병렬을 이루도록, 이후엔 반대로 원본 que_corpus 와 Augmentation된 ans_corpus 가 병렬을 이루도록 하여 전체 데이터가 원래의 3배가량으로 늘어나도록 

In [261]:
!pip install --upgrade gensim==3.8.3

In [262]:
import gensim
import random

def load_model(model_path):

    wv_model = gensim.models.Word2Vec.load(model_path)
   # wv_model= gensim.models.Word2Vec.load(model_path)
    return wv_model

In [263]:
print(question[:3])

['12시 땡', '1지망 학교 떨어졌어', '3박4일 놀러가고 싶다']


In [264]:
def lexical_sub(sentence, wv_model):
    """
    문장을 입력받아 Lexical Substitution을 수행하는 함수

    :param sentence: 대체를 수행할 문장
    :param wv_model: gensim Word2Vec 모델
    :return: 대체된 문장
    """
    tokens = sentence.split()
    sub_token = random.choice(tokens)
    #print(tokens)
    #print(sub_token)
    try:
        similar_word = wv_model.most_similar(sub_token)[0][0]
    except KeyError:
        similar_word = sub_token

    augmented_sentence = ' '.join([similar_word if token == sub_token else token for token in tokens])
    #print(augmented_sentence)
    return augmented_sentence

In [265]:
model_path = os.getenv('HOME') + '/aiffel/Aiffel_online_Quest/GoingDeeper06/ko.bin'
print(model_path)
wv_model = load_model(model_path)
print(lexical_sub(question[0],wv_model))
print(question[0])


/aiffel/aiffel/Aiffel_online_Quest/GoingDeeper06/ko.bin
12시 끗
12시 땡


/tmp/ipykernel_138/3238749839.py:14: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  similar_word = wv_model.most_similar(sub_token)[0][0]


In [266]:
from tqdm.notebook import tqdm

# 모델 파일 경로 (예: 'ko.bin')
model_path = os.getenv('HOME') + '/aiffel/Aiffel_online_Quest/GoingDeeper06/ko.bin'
print(model_path)
wv_model = load_model(model_path)

# 예제 문장
new_corpus_q = []
new_corpus_a = []

for q in tqdm(question):
    new_src = lexical_sub(q, wv_model)
    if new_src is not None: 
        new_corpus_q.append(new_src)
    # Augmentation이 없더라도 원본 문장을 포함시킵니다
    else: new_corpus_q.append(q)

for a in tqdm(answer):
    new_src = lexical_sub(a, wv_model)
    if new_src is not None: 
        new_corpus_a.append(new_src)
    # Augmentation이 없더라도 원본 문장을 포함시킵니다
    else: new_corpus_a.append(a)

#augmented_sentence = lexical_sub(sentence, wv_model)
print(new_corpus_a[:10])
print(new_corpus_q[:10])

/aiffel/aiffel/Aiffel_online_Quest/GoingDeeper06/ko.bin


  0%|          | 0/7712 [00:00<?, ?it/s]

/tmp/ipykernel_138/3238749839.py:14: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  similar_word = wv_model.most_similar(sub_token)[0][0]


  0%|          | 0/7712 [00:00<?, ?it/s]

['하루가 각기 가네요', '위로해 드립니다', '여행은 언제나 좋죠', '눈살이 찌푸려지죠', '다시 새로이 사는 게 마음 편해요', '많이 모르고 있을 수도 있어요', '시간을 정하고 해보세요', '자랑하는 자리니까요', '그 사람도 그럴 거예요', '혼자를 즐기세요']
['12시 끗', '1지망 학교 떨어졌어', '3박4일 놀러가고 싶다', 'ppl 심하네', 'sd카드 망가졌어', 'sns 맞팔 과연 안하지ㅠㅠ', 'sns 시간낭비인 거 아는데 매일 하는 중', 'sns보면 나만 빼고 다 행복해보여', '이따금 궁금해', '가끔은 혼자인게 좋다']


In [267]:
q_data_aug, a_data_aug = build_corpus(new_corpus_q,new_corpus_a)
print(q_data_aug[0])
print(a_data_aug[0])

['12', '시', '끗']
['하루', '가', '각기', '가', '네요']


Step 5. 데이터 벡터화
- 타겟 데이터인 ans_corpus 에 <start> 토큰과 <end> 토큰이 추가되지 않은 상태이니 이를 먼저 해결한 후 벡터화
- 특수 토큰을 더함으로써 ans_corpus 또한 완성이 되었으니, que_corpus 와 결합하여 전체 데이터에 대한 단어 사전을 구축하고 벡터화하여 enc_train 과 dec_train 을 얻으세요!

In [268]:
def add_token(ans_corpus):
    return ["<start>"] + ans_corpus + ["<end>"]

a_data = [add_token(a) for a in a_data]
new_corpus_a = [add_token(a) for a in a_data_aug]

In [269]:
ans_corpus = a_data + new_corpus_a +new_corpus_a
que_corpus = q_data + new_corpus_q+q_data

ans_cor = ans_corpus[:20000]
que_cor = que_corpus[:20000]

In [270]:
print(len(ans_corpus[:20000]))
print(len(que_corpus[:20000]))

20000
20000


In [271]:
def generate_tokenizer(corpus,
                       vocab_size,
                       lang="spa-eng",
                       pad_id=0,   # pad token의 일련번호
                       bos_id=1,  # 문장의 시작을 의미하는 bos token(<s>)의 일련번호
                       eos_id=2,  # 문장의 끝을 의미하는 eos token(</s>)의 일련번호
                       unk_id=3):   # unk token의 일련번호
    file = "./%s_corpus.txt" % lang
    model = "%s_spm" % lang

    with open(file, 'w') as f:
        for row in corpus: f.write(str(row) + '\n')

    import sentencepiece as spm
    spm.SentencePieceTrainer.Train(
        '--input=./%s --model_prefix=%s --vocab_size=%d'\
        % (file, model, vocab_size) + \
        '--pad_id==%d --bos_id=%d --eos_id=%d --unk_id=%d'\
        % (pad_id, bos_id, eos_id, unk_id)
    )

    tokenizer = spm.SentencePieceProcessor()
    tokenizer.Load('%s.model' % model)

    return tokenizer

In [272]:
que_cor = [' '.join(q) for q in que_cor]
ans_cor = [' '.join(a) for a in ans_cor]

print(que_cor[0])
print(ans_cor[0])

12 시 땡
<start> 하루 가 또 가 네요 <end>


In [277]:
VOCAB_SIZE = 3000
tokenizer = generate_tokenizer(que_cor + ans_cor, VOCAB_SIZE, 'kor-kor')
tokenizer.set_encode_extra_options("bos:eos")  # 문장 양 끝에 <s> , </s> 추가

sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=././kor-kor_corpus.txt --model_prefix=kor-kor_spm --vocab_size=3000--pad_id==0 --bos_id=1 --eos_id=2 --unk_id=3
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: ././kor-kor_corpus.txt
  input_format: 
  model_prefix: kor-kor_spm
  model_type: UNIGRAM
  vocab_size: 3000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 3
  bos_id: 1
  eos_id: 2
  pad_id: -1
 

True

es: 1
  escape_whitespaces: 1
  normalization_rule_tsv: 
}
denormalizer_spec {}
trainer_interface.cc(329) LOG(INFO) SentenceIterator is not specified. Using MultiFileSentenceIterator.
trainer_interface.cc(178) LOG(INFO) Loading corpus: ././kor-kor_corpus.txt
trainer_interface.cc(385) LOG(INFO) Loaded all 40000 sentences
trainer_interface.cc(400) LOG(INFO) Adding meta_piece: <s>
trainer_interface.cc(400) LOG(INFO) Adding meta_piece: </s>
trainer_interface.cc(400) LOG(INFO) Adding meta_piece: <unk>
trainer_interface.cc(405) LOG(INFO) Normalizing sentences...
trainer_interface.cc(466) LOG(INFO) all chars count=1042591
trainer_interface.cc(477) LOG(INFO) Done: 99.9503% characters are covered.
trainer_interface.cc(487) LOG(INFO) Alphabet size=1021
trainer_interface.cc(488) LOG(INFO) Final character coverage=0.999503
trainer_interface.cc(520) LOG(INFO) Done! preprocessed 40000 sentences.
unigram_model_trainer.cc(139) LOG(INFO) Making suffix array...
unigram_model_trainer.cc(143) LOG(INFO) Ex

In [278]:
test_sentence_count = int(len(que_cor)*0.05)
print("Test Size: ", test_sentence_count)
print("\n")

#train_que_cor = que_cor[:-test_sentence_count]
test_que_cor = que_cor[-test_sentence_count:]
print("Train Example:", len(que_cor))
for sen in que_cor[0:100][::20]: 
    print(">>", sen)
print("\n")
print("Test Example:", len(test_que_cor))
for sen in test_que_cor[0:100][::20]: 
    print(">>", sen)

Test Size:  1000


Train Example: 20000
>> 12 시 땡
>> 가족 들 보 고 싶 어
>> 갑자기 물어봐서 당황 했 어
>> 개 좋 아
>> 게으른 동료 가 있 어


Test Example: 1000
>> 이별 너무 힘드 네 ㅠ
>> 이 별 한 달 반 째
>> 이별 후 후회 아쉬움
>> 이별 당시 보다 더 짠 한 건 나 떠났 으면 잘 이나 살 지
>> 이별 은 진짜 비겁 한 짓 같 아


In [279]:
# 토큰화 함수
def make_corpus(sentences, tokenizer):
    corpus = []
    for sentence in tqdm(sentences):
        tokens = tokenizer.encode_as_ids(sentence)
        corpus.append(tokens)
    return corpus


In [280]:
# 영어와 스페인어를 각각 토큰화 해줍니다. 훈련 데이터만 토큰화를 하고, 같은 토크나이저를 사용한다는 점
eng_corpus = make_corpus(que_cor, tokenizer)
spa_corpus = make_corpus(ans_cor, tokenizer)

  0%|          | 0/20000 [00:00<?, ?it/s]

  0%|          | 0/20000 [00:00<?, ?it/s]

In [281]:
# 토큰화 잘 됐는지 확인
print(eng_corpus[0])
print('\n')
print(spa_corpus[0])

[1, 330, 1882, 119, 1851, 2]


[1, 5, 6, 0, 8, 10, 0, 4, 440, 18, 199, 18, 54, 5, 11, 7, 9, 4, 2]


In [282]:
MAX_LEN = 50
enc_ndarray = tf.keras.preprocessing.sequence.pad_sequences(eng_corpus, maxlen=MAX_LEN, padding='post')
dec_ndarray = tf.keras.preprocessing.sequence.pad_sequences(spa_corpus, maxlen=MAX_LEN, padding='post')

print(enc_ndarray[0])
print(dec_ndarray[0])

[   1  330 1882  119 1851    2    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0]
[  1   5   6   0   8  10   0   4 440  18 199  18  54   5  11   7   9   4
   2   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0]


In [283]:
BATCH_SIZE = 64
train_dataset = tf.data.Dataset.from_tensor_slices((enc_ndarray, dec_ndarray)).batch(batch_size=BATCH_SIZE)


Step 6. 훈련하기

In [284]:
# Positional Encoding 구현
def positional_encoding(pos, d_model):
    # 각 모델에 홀수 짝수에 각각 정현파 함수를 적용한다 sin, cos
    # 위치 별 벡터 구하는 공식
    def cal_angle(position,i):
        # postion, index
        return position / np.power(10000,(2*(i//2) / np.float32(d_model)))
    
    def get_posi_engle_vec(position):
        return [cal_angle(position, i) for i in range(d_model)]

    # 모든 토큰에 position 값을 계산하고 나서 array로 만든다
    sinusoid_table = np.array([get_posi_engle_vec(pos_i) for pos_i in range(pos)])

    # 짝수 - sin , 홀수 - cos
    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])

    return sinusoid_table

In [285]:
# Mask  생성하기
def generate_padding_mask(seq):
    # seq == 0 이면 float32로 바꾸고
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]

def generate_lookahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask

def generate_masks(src, tgt):
    enc_mask = generate_padding_mask(src)
    dec_enc_mask = generate_padding_mask(src)

    dec_lookahead_mask = generate_lookahead_mask(tgt.shape[1])
    dec_tgt_padding_mask = generate_padding_mask(tgt)
    dec_mask = tf.maximum(dec_tgt_padding_mask, dec_lookahead_mask)

    return enc_mask, dec_enc_mask, dec_mask


In [286]:

class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        
        self.depth = d_model // self.num_heads
        
        self.W_q = tf.keras.layers.Dense(d_model)
        self.W_k = tf.keras.layers.Dense(d_model)
        self.W_v = tf.keras.layers.Dense(d_model)
        
        self.linear = tf.keras.layers.Dense(d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask):
        d_k = tf.cast(K.shape[-1], tf.float32)
        QK = tf.matmul(Q, K, transpose_b=True)

        scaled_qk = QK / tf.math.sqrt(d_k)

        if mask is not None: scaled_qk += (mask * -1e9)  

        attentions = tf.nn.softmax(scaled_qk, axis=-1)
        out = tf.matmul(attentions, V)

        return out, attentions
        

    def split_heads(self, x):
        bsz = x.shape[0]
        split_x = tf.reshape(x, (bsz, -1, self.num_heads, self.depth))
        split_x = tf.transpose(split_x, perm=[0, 2, 1, 3])

        return split_x

    def combine_heads(self, x):
        bsz = x.shape[0]
        combined_x = tf.transpose(x, perm=[0, 2, 1, 3])
        combined_x = tf.reshape(combined_x, (bsz, -1, self.d_model))

        return combined_x

    
    def call(self, Q, K, V, mask):
        # 각각 Dense layer를 거쳐서 상태가 있는 형태로 만든다
        WQ = self.W_q(Q)
        WK = self.W_k(K)
        WV = self.W_v(V)
        
        # Q K V 값을 동일하게 num head 만큼 나눠준다
        WQ_splits = self.split_heads(WQ)
        WK_splits = self.split_heads(WK)
        WV_splits = self.split_heads(WV)


        # 그리고 attention layer를 거친다.
        out, attention_weights = self.scaled_dot_product_attention(
            WQ_splits, WK_splits, WV_splits, mask)
                        
        out = self.combine_heads(out)
        out = self.linear(out)
            
        return out, attention_weights

In [287]:
# Position-wise Feed Forward Network 구현
class PoswiseFeedForwardNet(tf.keras.layers.Layer):
    def __init__(self, d_model, d_ff):
        super(PoswiseFeedForwardNet, self).__init__()
        self.d_model = d_model
        self.d_ff = d_ff

        self.fc1 = tf.keras.layers.Dense(d_ff, activation='relu')
        self.fc2 = tf.keras.layers.Dense(d_model)

    def call(self, x):

        x = self.fc1(x)
        out = self.fc2(x)
        
        return out


In [288]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()

        self.enc_self_attn = MultiHeadAttention(d_model, n_heads)
        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)
        
    def call(self, x, mask):
        '''
        Multi-Head Attention
        '''
        # 잔차
        residual = x
        # 정규화
        out = self.norm_1(x)
        # self attention
        out, enc_attn = self.enc_self_attn(out, out, out, mask)
        # dropout
        out = self.do(out)
        # residual 
        out += residual
        
        '''
        Position-Wise Feed Forward Network
        '''
        # 잔차
        residual = out
        # attention에서 나온 값 layer norm
        out = self.norm_2(out)
        # feed forward net
        out = self.ffn(out)
        # drop out
        out = self.do(out)
        out += residual
        
        return out, enc_attn

In [289]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()

        self.dec_self_attn = MultiHeadAttention(d_model, num_heads)
        self.enc_dec_attn = MultiHeadAttention(d_model, num_heads)

        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)
    
    def call(self, x, enc_out, dec_enc_mask, padding_mask):
        '''
        Masked Multi-Head Attention
        '''
        # 첫번 째 self attention
        residual = x
        out = self.norm_1(x)
        out, dec_attn = self.dec_self_attn(out, out, out, padding_mask)
        out = self.do(out)
        out += residual

        '''
        Multi-Head Attention
        '''
        # encoder의 Value 값으로 Attention
        residual = out
        out = self.norm_2(out)
        # Q, K, V 순서에 주의하세요!
        out, dec_enc_attn = self.enc_dec_attn(Q=out, K=enc_out, V=enc_out, mask=dec_enc_mask)
        out = self.do(out)
        out += residual
        
        '''
        Position-Wise Feed Forward Network
        '''
        # feed forward net
        residual = out
        out = self.norm_3(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual

        return out, dec_attn, dec_enc_attn

In [290]:
class Encoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Encoder, self).__init__()
        self.n_layers = n_layers
        self.enc_layers = [EncoderLayer(d_model, n_heads, d_ff, dropout) 
                        for _ in range(n_layers)]
    
        self.do = tf.keras.layers.Dropout(dropout)
        
    def call(self, x, mask):
        out = x
    
        enc_attns = list()
        for i in range(self.n_layers):
            out, enc_attn = self.enc_layers[i](out, mask)
            enc_attns.append(enc_attn)
        
        return out, enc_attns

In [291]:
class Decoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Decoder, self).__init__()
        self.n_layers = n_layers
        self.dec_layers = [DecoderLayer(d_model, n_heads, d_ff, dropout) 
                            for _ in range(n_layers)]
                            
    def call(self, x, enc_out, dec_enc_mask, padding_mask):
        out = x
    
        dec_attns = list()
        dec_enc_attns = list()
        for i in range(self.n_layers):
            out, dec_attn, dec_enc_attn = \
            self.dec_layers[i](out, enc_out, dec_enc_mask, padding_mask)

            dec_attns.append(dec_attn)
            dec_enc_attns.append(dec_enc_attn)

        return out, dec_attns, dec_enc_attns

In [292]:
class Transformer(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    src_vocab_size,
                    tgt_vocab_size,
                    pos_len,
                    dropout=0.2,
                    shared_fc=True,
                    shared_emb=False):
        super(Transformer, self).__init__()
        
        self.d_model = tf.cast(d_model, tf.float32)

        if shared_emb:
            self.enc_emb = self.dec_emb = \
            tf.keras.layers.Embedding(src_vocab_size, d_model)
        else:
            self.enc_emb = tf.keras.layers.Embedding(src_vocab_size, d_model)
            self.dec_emb = tf.keras.layers.Embedding(tgt_vocab_size, d_model)

        self.pos_encoding = positional_encoding(pos_len, d_model)
        self.do = tf.keras.layers.Dropout(dropout)

        self.encoder = Encoder(n_layers, d_model, n_heads, d_ff, dropout)
        self.decoder = Decoder(n_layers, d_model, n_heads, d_ff, dropout)

        self.fc = tf.keras.layers.Dense(tgt_vocab_size)

        self.shared_fc = shared_fc

        if shared_fc:
            self.fc.set_weights(tf.transpose(self.dec_emb.weights))

    def embedding(self, emb, x):
        seq_len = x.shape[1]

        out = emb(x)

        if self.shared_fc: out *= tf.math.sqrt(self.d_model)

        out += self.pos_encoding[np.newaxis, ...][:, :seq_len, :]
        out = self.do(out)

        return out

        
    def call(self, enc_in, dec_in, enc_mask, dec_enc_mask, dec_mask):
        enc_in = self.embedding(self.enc_emb, enc_in)
        dec_in = self.embedding(self.dec_emb, dec_in)

        enc_out, enc_attns = self.encoder(enc_in, enc_mask)
        
        dec_out, dec_attns, dec_enc_attns = \
        self.decoder(dec_in, enc_out, dec_enc_mask, dec_mask)
        
        logits = self.fc(dec_out)
        
        return logits, enc_attns, dec_attns, dec_enc_attns

In [293]:
transformer = Transformer(
    n_layers=2,
    d_model=512,
    n_heads=8,
    d_ff=2048,
    src_vocab_size=VOCAB_SIZE,
    tgt_vocab_size=VOCAB_SIZE,
    pos_len=200,
    dropout=0.3,
    shared_fc=True,
    shared_emb=True)

d_model = 512

In [294]:

class LearningRateScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(LearningRateScheduler, self).__init__()
        
        self.d_model = d_model
        self.warmup_steps = warmup_steps
    
    def __call__(self, step):
        arg1 = step ** -0.5
        arg2 = step * (self.warmup_steps ** -1.5)
        
        return (self.d_model ** -0.5) * tf.math.minimum(arg1, arg2)

In [295]:
# Learning Rate 인스턴스 선언 & Optimizer 구현

learning_rate = LearningRateScheduler(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate,
                                        beta_1=0.9,
                                        beta_2=0.98, 
                                        epsilon=1e-9)

In [296]:
# Loss Function 정의
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

In [297]:

@tf.function()
def train_step(src, tgt, model, optimizer):
    tgt_in = tgt[:, :-1]  # Decoder의 input
    gold = tgt[:, 1:]     # Decoder의 output과 비교하기 위해 right shift를 통해 생성한 최종 타겟

    enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt_in)

    with tf.GradientTape() as tape:
        predictions, enc_attns, dec_attns, dec_enc_attns = \
        model(src, tgt_in, enc_mask, dec_enc_mask, dec_mask)
        loss = loss_function(gold, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return loss, enc_attns, dec_attns, dec_enc_attns

In [332]:
EPOCHS=30

for epoch in range(EPOCHS):
    total_loss = 0
    
    
    dataset_count = tf.data.experimental.cardinality(train_dataset).numpy()
    tqdm_bar = tqdm(enumerate(train_dataset), total=dataset_count)
    for (batch, (src, tgt)) in tqdm_bar:
        batch_loss, enc_attns, dec_attns, dec_enc_attns = train_step(src, tgt, transformer, optimizer)
        total_loss += batch_loss

        # 여기서 tqdm_bar.set_description을 사용하여 에폭과 배치 손실을 출력할 수 있습니다.
        tqdm_bar.set_description(f"Epoch {epoch + 1} Batch {batch} Loss {batch_loss.numpy():.4f}")

    # 에폭마다 평균 손실을 계산합니다.
    total_loss /= dataset_count
    print(f"Epoch {epoch + 1} Loss {total_loss:.4f}")


  0%|          | 0/313 [00:00<?, ?it/s]

Epoch 1 Loss 0.4108


  0%|          | 0/313 [00:00<?, ?it/s]

Epoch 2 Loss 0.3918


  0%|          | 0/313 [00:00<?, ?it/s]

...

Epoch 28 Loss 0.0494


  0%|          | 0/313 [00:00<?, ?it/s]

Epoch 29 Loss 0.0497


  0%|          | 0/313 [00:00<?, ?it/s]

Epoch 30 Loss 0.0476


Step 7. 성능 측정하기
- 챗봇의 경우, 올바른 대답을 하는지가 중요한 평가 지표
- BLEU Score를 계산하는 calculate_bleu() 함수

In [301]:
# evaluation
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

# 아래 두 문장을 바꿔가며 테스트 해보세요
reference = "하루가 참 길어요".split()
candidate = "하루 가 참 길 어 요".split()

print("질문:", reference)
print("대답:", candidate)
print("BLEU Score:", sentence_bleu([reference], candidate))

질문: ['하루가', '참', '길어요']
대답: ['하루', '가', '참', '길', '어', '요']
BLEU Score: 1.1640469867513693e-231


/opt/conda/lib/python3.9/site-packages/nltk/translate/bleu_score.py:515: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/opt/conda/lib/python3.9/site-packages/nltk/translate/bleu_score.py:515: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/opt/conda/lib/python3.9/site-packages/nltk/translate/bleu_score.py:515: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg

In [302]:
def calculate_bleu(reference, candidate, weights=[0.25, 0.25, 0.25, 0.25]):
    return sentence_bleu([reference],
                         candidate,
                         weights=weights,
                         smoothing_function=SmoothingFunction().method1)  # smoothing_function 적용

print("BLEU-1:", calculate_bleu(reference, candidate, weights=[1, 0, 0, 0]))
print("BLEU-2:", calculate_bleu(reference, candidate, weights=[0, 1, 0, 0]))
print("BLEU-3:", calculate_bleu(reference, candidate, weights=[0, 0, 1, 0]))
print("BLEU-4:", calculate_bleu(reference, candidate, weights=[0, 0, 0, 1]))

print("\nBLEU-Total:", calculate_bleu(reference, candidate))

BLEU-1: 0.16666666666666669
BLEU-2: 0.02
BLEU-3: 0.025
BLEU-4: 0.03333333333333333

BLEU-Total: 0.040824829046386304


In [333]:
def translate(tokens, model, src_tokenizer, tgt_tokenizer):
    padded_tokens = tf.keras.preprocessing.sequence.pad_sequences([tokens],
                                                           maxlen=MAX_LEN,
                                                           padding='post')
    ids = []
    output = tf.expand_dims([tgt_tokenizer.bos_id()], 0)   
    for i in range(MAX_LEN):
        enc_padding_mask, combined_mask, dec_padding_mask = \
        generate_masks(padded_tokens, output)

        predictions, _, _, _ = model(padded_tokens, 
                                      output,
                                      enc_padding_mask,
                                      combined_mask,
                                      dec_padding_mask)

        predicted_id = \
        tf.argmax(tf.math.softmax(predictions, axis=-1)[0, -1]).numpy().item()

        if tgt_tokenizer.eos_id() == predicted_id:
            result = tgt_tokenizer.decode_ids(ids)  
            return result

        ids.append(predicted_id)
        output = tf.concat([output, tf.expand_dims([predicted_id], 0)], axis=-1)

    result = tgt_tokenizer.decode_ids(ids)  
    return result


In [334]:
def eval_bleu_single(model, src_sentence, tgt_sentence, src_tokenizer, tgt_tokenizer, verbose=True):
    src_tokens = src_tokenizer.encode_as_ids(src_sentence)
    tgt_tokens = tgt_tokenizer.encode_as_ids(tgt_sentence)

    if (len(src_tokens) > MAX_LEN): return None
    if (len(tgt_tokens) > MAX_LEN): return None

    reference = tgt_sentence.split()
    candidate = translate(src_tokens, model, src_tokenizer, tgt_tokenizer).split()

    score = sentence_bleu([reference], candidate,
                          smoothing_function=SmoothingFunction().method1)

    if verbose:
        print("Source Sentence: ", src_sentence)
        print("Model Prediction: ", ' '.join(candidate))
        print("Real: ", reference)
        print("Score: %lf\n" % score)
        print("-----------------------------------------------------")
        
    return score

In [335]:
# evaluation
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
# Q. 인덱스를 바꿔가며 테스트해 보세요
test_idx = range(10,15)

for test in test_idx:
    eval_bleu_single(transformer, 
                     que_cor[test], 
                     ans_cor[test], 
                     tokenizer, 
                     tokenizer)

Source Sentence:  가난 한 자 의 설움
Model Prediction:  <sssnsns 종교 문제 가 되 기 힘들 죠 <end>
Real:  ['<start>', '돈', '은', '다시', '들어올', '거', '예요', '<end>']
Score: 0.023980

-----------------------------------------------------
Source Sentence:  가만 있 어도 땀 난다
Model Prediction:  <ssssssns 끊 미리 충전 도 되 어 봐요 <end>
Real:  ['<start>', '땀', '을', '식혀', '주', '세요', '<end>']
Score: 0.023980

-----------------------------------------------------
Source Sentence:  가상 화폐 쫄딱 망함
Model Prediction:  <sssssss 에너찬 가 필요 한 순간 이 네요 <end>
Real:  ['<start>', '어서', '잊', '고', '새', '출발', '하', '세요', '<end>']
Score: 0.023980

-----------------------------------------------------
Source Sentence:  가스 불 켜 고 나갔 어
Model Prediction:  <sssssn 자책 하 지 마세요 <end>
Real:  ['<start>', '빨리', '집', '에', '돌아가', '서', '끄', '고', '나오', '세요', '<end>']
Score: 0.017742

-----------------------------------------------------
Source Sentence:  가스 비 너무 많이 나왔 다
Model Prediction:  <sssssnsns 검색 하 지 마세요 <end>
Real:  ['<start>', '다음', '달', '에', '는', '더', '절약', 

In [336]:
def eval_bleu(model, src_sentences, tgt_sentence, src_tokenizer, tgt_tokenizer, verbose=True):
    total_score = 0.0
    sample_size = len(src_sentences)
    
    for idx in tqdm(range(sample_size)):
        score = eval_bleu_single(model, src_sentences[idx], tgt_sentence[idx], src_tokenizer, tgt_tokenizer, verbose)
        if not score: continue
        
        total_score += score
    
    print("Num of Sample:", sample_size)
    print("Total Score:", total_score / sample_size)
    

In [337]:
eval_bleu(transformer, que_cor[:100], ans_cor[:100], tokenizer, tokenizer, verbose=False)

  0%|          | 0/100 [00:00<?, ?it/s]

Num of Sample: 100
Total Score: 0.08239454782465133


### 회고
- 지난 과정에 이어서 트랜스포머로 번역기를 만들어보았다.생각보다 전처리 부분에서 시간이 많이 걸리고 중요하다는 것을 느꼈다.
- 데이터 증강과 BLUE에 대해서 자세히는 모르고 있었는데 배울 수 있는 좋은 프로젝트였다.